# Import Library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import to_networkx, subgraph, k_hop_subgraph
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, GINConv
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import random

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load Cora Dataset

In [ ]:
dataset = Planetoid(root="../data/Cora", name="Cora")
data = dataset[0]

In [ ]:
node_of_interest = random.randint(0, data.num_nodes - 1)
print("Node of Interest:", node_of_interest)

# extract 2-hop subgraph
subset_nodes, sub_edge_index, _, _ = k_hop_subgraph(
    node_of_interest, 
    num_hops=2, 
    edge_index=data.edge_index, 
    relabel_nodes=True
)

sub_x = data.x[subset_nodes]
sub_y = data.y[subset_nodes]

print("Subgraph Nodes:", sub_x.size(0))

In [ ]:
# G = to_networkx(
#     Data(x=sub_x, edge_index=sub_edge_index), 
#     to_undirected=True
# )

# plt.figure(figsize=(6,6))
# nx.draw(G, node_size=50)
# plt.title("Subgraph (2-hop Neighborhood)")
# plt.show()

In [ ]:
#Jumlah Node
data.num_nodes

In [ ]:
#jumlah edge
data.num_edges

In [ ]:
#dimensi fitur
data.num_features

In [ ]:
#jumlah keasl
dataset.num_classes

In [ ]:
#nama kelas
data.y.unique()

In [ ]:
#nodes belong to classes
data.y.bincount()

In [ ]:
class_counts = data.y.bincount()
for class_idx in range(len(class_counts)):
    print(f"Kelas {class_idx}: {class_counts[class_idx].item()} nodes")

In [ ]:
class_counts = data.y.bincount()
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot of class distribution
axes[0].bar(range(len(class_counts)), class_counts.numpy())
axes[0].set_xlabel('Class')
axes[0].set_ylabel('Number of Nodes')
axes[0].set_title('Class Distribution in Cora Dataset')
axes[0].grid(True, alpha=0.3)

# Pie chart of class distribution
colors = plt.cm.Set3(np.linspace(0, 1, len(class_counts)))
axes[1].pie(class_counts.numpy(), labels=[f'Class {i}' for i in range(len(class_counts))], 
            autopct='%1.1f%%', colors=colors)
axes[1].set_title('Class Distribution (Percentage)')

plt.tight_layout()
plt.show()

In [ ]:
#train nodes
data.train_mask.sum().item()

In [ ]:
print(f"\nTrain ratio: {data.train_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
#validation nodes
data.val_mask.sum().item()

In [ ]:
print(f"Validation ratio: {data.val_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
#test nodes
data.test_mask.sum().item()

In [ ]:
print(f"Test ratio: {data.test_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
data.edge_index.shape

In [ ]:
data.edge_index[:, :5]

In [ ]:
data.x[0, :10]

In [ ]:
node_of_interest = 100  # Fixed for consistent visualization
print(f"Node of Interest: {node_of_interest} (Class: {data.y[node_of_interest].item()})")

In [ ]:
# Extract 2-hop subgraph
subset_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
    node_of_interest, 
    num_hops=2, 
    edge_index=data.edge_index, 
    relabel_nodes=True,
    num_nodes=data.num_nodes
)

sub_x = data.x[subset_nodes]
sub_y = data.y[subset_nodes]

print(f"Subgraph contains {sub_x.size(0)} nodes")
print(f"Classes in subgraph: {torch.unique(sub_y).tolist()}")

In [ ]:
# Convert to NetworkX for visualization
subgraph_data = Data(x=sub_x, edge_index=sub_edge_index)
G = to_networkx(subgraph_data, to_undirected=True)

plt.figure(figsize=(12, 10))

# Create color map for classes
cmap = plt.cm.tab10
node_colors = [cmap(y) for y in sub_y.numpy()]

# Get positions using spring layout
pos = nx.spring_layout(G, seed=42, k=1)

# Highlight the center node
center_idx = (subset_nodes == node_of_interest).nonzero(as_tuple=True)[0].item()

# Draw the graph
nx.draw_networkx_nodes(G, pos, node_size=300, 
                       node_color=node_colors, 
                       alpha=0.8, 
                       linewidths=0.5,
                       edgecolors='black')
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5, edge_color='gray')

# Highlight center node
nx.draw_networkx_nodes(G, pos, nodelist=[center_idx], 
                       node_size=500, 
                       node_color='red',
                       edgecolors='black', 
                       linewidths=2)

# Add labels for the center node
labels = {center_idx: 'Center'}
nx.draw_networkx_labels(G, pos, labels, font_size=12, font_weight='bold')

plt.title(f"2-hop Neighborhood of Node {node_of_interest}\n(Class {data.y[node_of_interest].item()})", fontsize=14)
plt.axis('off')

In [ ]:
# from matplotlib.patches import Patch
# legend_elements = [Patch(facecolor=cmap(i), label=f'Class {i}') 
#                    for i in range(dataset.num_classes)]
# plt.legend(handles=legend_elements, loc='upper right', title="Classes")

# plt.tight_layout()
# plt.show()

In [ ]:
degrees = np.zeros(data.num_nodes)
for i in range(data.num_nodes):
    degrees[i] = (data.edge_index[0] == i).sum().item()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of degree distribution
axes[0].hist(degrees, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Degree')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Degree Distribution Histogram')
axes[0].grid(True, alpha=0.3)

# Log-log plot for power law check
unique_degrees, degree_counts = np.unique(degrees, return_counts=True)
axes[1].loglog(unique_degrees, degree_counts, 'bo', alpha=0.7)
axes[1].set_xlabel('Degree (log)')
axes[1].set_ylabel('Frequency (log)')
axes[1].set_title('Degree Distribution (Log-Log Scale)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print(f"• Average degree: {degrees.mean():.2f}")
print(f"• Maximum degree: {degrees.max():.0f}")
print(f"• Minimum degree: {degrees.min():.0f}")

# PRA-PROSES (Message Passing Layer)

$h_u^{(k)} = \sum W_{\text{msg}} \cdot h_u^{(k-1)} + W_{\text{self}} \cdot h_v^{(k-1)}$

 di mana,
 1. **k** adalah urutan lapisan GNN.
 2. **Wmsg** adalah bobot yang diakses bersama dengan neighborhood nodes.
 3. **Wself** adalah bobot milik node asal (v).

In [ ]:
class CustomMessagePassing(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.W_msg = nn.Linear(in_channels, out_channels)  # For neighbors
        self.W_self = nn.Linear(in_channels, out_channels) # For self
        
    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)
    
    def message(self, x_j):
        return self.W_msg(x_j)
    
    def update(self, aggr_out, x):
        return aggr_out + self.W_self(x)

In [ ]:
# class CustomMessagePassing(MessagePassing):
#     def __init__(self, in_channels, out_channels, aggr='add'):
#         super().__init__(aggr=aggr)
#         self.W_msg = nn.Linear(in_channels, out_channels)
#         self.W_self = nn.Linear(in_channels, out_channels)
        
#     def forward(self, x, edge_index):
#         return self.propagate(edge_index, x=x)
    
#     def message(self, x_j):
#         return self.W_msg(x_j)
    
#     def update(self, aggr_out, x):
#         return aggr_out + self.W_self(x)

In [ ]:
# TEST MESSAGE PASSING

test_mp = CustomMessagePassing(in_channels=16, out_channels=32)
test_mp

In [ ]:
test_mp.W_msg.weight.shape

In [ ]:
test_mp.W_self.weight.shape

# Intra-layer GNN Block

$$
\mathbf{h}_v^{(k)} =
\text{ACT}\!\left(
    \text{Dropout}\!\left(
        \text{BatchNorm}\!\left(
            \mathbf{W}^{(k-1)}\,\mathbf{h}_v^{(k-1)} + \mathbf{b}^{(k-1)}
        \right)
    \right)
\right)
+ \mathbf{h}_v^{(k-1)}
$$


di mana:

1. **ACT** adalah fungsi aktivasi yang Anda pilih.
2. **BatchNorm** adalah `1D-Batch Normalization`.


## DEFINE SINGLE GNN LAYER 

In [ ]:
class GNNLayer(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.5):
        super().__init__()
        
        self.message_passing = CustomMessagePassing(in_channels, out_channels)
        
        # Post-message-passing processing
        self.linear = nn.Linear(out_channels, out_channels)
        self.batch_norm = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()
        
        # Residual connection
        if in_channels != out_channels:
            self.residual = nn.Linear(in_channels, out_channels)
        else:
            self.residual = nn.Identity()
    
    def forward(self, x, edge_index):
        x_input = x
        
        # Message passing
        x_mp = self.message_passing(x, edge_index)
        
        # Process features
        x_linear = self.linear(x_mp)
        x_bn = self.batch_norm(x_linear)
        x_drop = self.dropout(x_bn)
        x_act = self.activation(x_drop)
        
        # Residual connection
        x_residual = self.residual(x_input)
        return x_act + x_residual

## Test GNN Layer

In [ ]:
test_layer = GNNLayer(in_channels=16, out_channels=32)
test_layer

In [ ]:
test_layer.batch_norm

In [ ]:
test_layer.dropout.p

In [ ]:
test_layer.activation

# Full Model with k>1 Layers

## Build Model

In [ ]:
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes, num_layers, dropout=0.6):
        super().__init__()
        
        self.input_proj = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout)  # Increase dropout
        )
        
        self.layers = nn.ModuleList()
        for i in range(num_layers - 1):
            self.layers.append(
                GCNConv(hidden_channels, hidden_channels)
            )
        
        self.layers.append(
            GCNConv(hidden_channels, out_channels)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(out_channels, num_classes),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x, edge_index):
        x = self.input_proj(x)
        
        for i, layer in enumerate(self.layers):
            x = layer(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.6, training=self.training)  # Increase dropout
        
        return self.classifier(x)

## Initialize Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# model = GNNModel(
#     in_channels=data.num_features,
#     hidden_channels=512,
#     out_channels=dataset.num_classes
# ).to(device)

# print(f"Model created on {device}")

In [ ]:
# model = GNNModel(
#     in_channels=data.num_features,
#     hidden_channels=256,
#     out_channels=dataset.num_classes
# ).to(device)

In [ ]:
out_classes = dataset.num_classes

In [ ]:
model = GNNModel(
    in_channels=1433,
    hidden_channels=128,
    out_channels=128,
    num_classes=7,
    num_layers=2,  # Reduced from 4
    dropout=0.6
)

In [ ]:
# # Increase weight decay
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), 
                             lr=0.01, 
                             weight_decay=5e-4,  # L2 regularization
                             betas=(0.9, 0.999))

In [ ]:
# model.classifier = nn.Linear(128, out_classes).to(device)

In [ ]:
model.classifier = nn.Linear(128, out_classes).to(device)

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',
    factor=0.5,
    patience=20,
    min_lr=1e-5,
    verbose=True
)

## RAINING LOOP WITH VALIDATION LOSS

### DEFINE TRAINING STEP FUNCTION

In [ ]:
def train_one_epoch(model, data, optimizer, smoothing=0.1):
    model.train()
    optimizer.zero_grad()
    
    out = model(data.x, data.edge_index)
    
    # Label smoothing
    n_classes = out.size(1)
    target = data.y[data.train_mask]
    target_onehot = torch.zeros_like(out[data.train_mask]).scatter_(1, target.unsqueeze(1), 1)
    target_smoothed = target_onehot * (1 - smoothing) + smoothing / n_classes
    
    loss = F.kl_div(F.log_softmax(out[data.train_mask], dim=1), 
                   target_smoothed, 
                   reduction='batchmean')
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    
    return loss.item()

### DEFINE EVALUATION FUNCTION

In [ ]:
def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out[mask].argmax(dim=1)
        correct = (pred == data.y[mask]).sum().item()
        accuracy = correct / mask.sum().item()
    return accuracy

In [ ]:
data = data.to(device)

In [ ]:
# Define number of epochs
epochs = 300  # or whatever number you want

In [ ]:
# Also define other variables
patience = 100
train_losses = []
val_accuracies = []
test_accuracies = []
best_val_acc = 0.0
patience_counter = 0

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Move data
data = data.to(device)


In [ ]:
for epoch in range(epochs):
    # 1. Training
    loss = train_one_epoch(model, data, optimizer)
    train_losses.append(loss)
    
    # 2. Evaluation (compute val_acc and test_acc) - DO THIS FIRST
    val_acc = evaluate(model, data, data.val_mask)
    test_acc = evaluate(model, data, data.test_mask)
    
    val_accuracies.append(val_acc)
    test_accuracies.append(test_acc)
    
    # 3. Update scheduler - NOW val_acc is defined!
    scheduler.step(val_acc)  # ← Now this works
    
    # 4. Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), '../models/best_model.pth')
    else:
        patience_counter += 1
    
    # 5. Progress reporting
    if epoch % 10 == 0:
        lr = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch:3d}: Loss={loss:.4f}, Val={val_acc:.4f}, Test={test_acc:.4f}, LR={lr:.6f}')
    
    # 6. Early stopping condition
    if patience_counter >= patience:
        print(f'\nEarly stopping at epoch {epoch}')
        break

In [ ]:
# In training loop:
scheduler.step(val_acc)  # Pass validation accuracy

In [ ]:
# Load best model
model.load_state_dict(torch.load('../models/best_model.pth'))

In [ ]:
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index)
    
    # Calculate final accuracies
    train_pred = out[data.train_mask].argmax(dim=1)
    train_acc = (train_pred == data.y[data.train_mask]).float().mean().item()
    
    val_pred = out[data.val_mask].argmax(dim=1)
    val_acc = (val_pred == data.y[data.val_mask]).float().mean().item()
    
    test_pred = out[data.test_mask].argmax(dim=1)
    test_acc = (test_pred == data.y[data.test_mask]).float().mean().item()

In [ ]:
class SimpleGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)
        self.dropout = dropout
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = SimpleGCN(
    in_channels=1433,
    hidden_channels=16,  # Smaller hidden dimension
    num_classes=7,
    dropout=0.6
)